# <center/> MICRO-452 - Basics of Mobile Robotics 
### <center/> Thymio Robot Project
#### <center/> TEAM 12

***
__<p style='text-align: left;'> Group members & Sciper</p>__ 
<br/> Jacques Bénand 325957
<br/> Mathieu Schertenleib 313318
<br/> Théo Maetz 312583
<br/> Maria Cherchouri 311251
***

### Introduction

This Jupyter notebook provides all the information you need about the choices we've made for this project and the theory behind our implementation, to run the project directly, simply execute the main() cell before the difficulties encountered.

Remember that the aim was to control our Thymio robot by executing 4 separate components: vision, global navigation, local navigation, and filtering which we'll assemble later.  
After this reassembly, the Thymio must move from an arbitrary position in the map to a target that can be placed anywhere in the environment while avoiding the global obstacles set by the map. During its navigation, it must also be able to avoid local obstacles that may be placed on its path. 

We use ArUco markers to detect the four corners of the map, as well as the robot and the target. A visibility graph is used to implement the path from the robot to the goal while avoiding obstacles, Astolfi controller for control and Kalman filter for filtering. Finally, we'll use our robot's proximal sensors to avoid 3D local obstacles.

### Environment

The environment is a white background made by taping two A0 sheets of paper length-wise. This allows for a clear contrast from the fixed obstacle in black placed on top of the map, cut up from black sheets of paper as to obtain a desired shape. These obstacles are wilfully flat as to not trigger the proximity sensors of the Thymio, which are only to be triggered in the case of a surprise 3 dimensional obstacle and activate the local avoidance protocol. A camera is setup above the map in such a way that all 4 markers defining the map are visible in the frame. This  will function with any set up from the camera as long as these 4 markers are visible.

### Code structure

The folder src/ contains all the files used to run the project. Namely:
- **main.py** runs the whole system, manages the interface, the main control loop and calls the different submodules
- **parameters.py** stores the global constants needed throughout the different submodules
- **calibrate_camera.py** implements a simple interface for the user to calibrate the camera
- **threaded_capture.py** implements a video capture on its own thread
- **image_processing.py** implements all functions related to computer vision and coordinate transformations
- **global_map.py** manages the building and updating of the obstacle mask, the global map and pathfinding
- **kalman_filter.py** implements the Kalman filter algorithm
- **controller.py** implements our (slightly modified) Astolfi controller
- **local_navigation.py** implements local obstacle detection and avoidance

The folder tests/ contains various tests done during the development of this project. Most of them can be run independently, and they might be unit tests (for example for segment intersection), benchmarks or higher level programs, like the one included in this notebook for interactive map visualization.

### Libraries used

- [tdmclient](https://pypi.org/project/tdmclient/) for connecting to the robot
- [OpenCV](https://opencv.org/) for image processing and computer vision
- [NumPy](https://numpy.org/) for general array processing and linear algebra
- [DearPyGui](https://github.com/hoffstadt/DearPyGui) for the user interface

### Overall implementation and structure

The flow chart of the overall global implementation of the project is provided in the figure below

![](images/flow_chart.png)

### I. Vision

* ##### **ArUco Markers**
     
     
     To determine the position and orientation of the Thymio, it is of the utmost importance to determine the corresponding point in our real-life environment to a point in our 2D image projection. This task is facilitated by the use of ArUco markers generated directly from the OpenCV library. Their respectively unique black and white patterns are optimized as to make their detection as fast as possible whilst easily differentiating them.

* ##### **Camera Calibration using ArUco ChessBoard**
     
     
     When calibrating the camera, an ArUco chessboard ChArUco is generated similar to this one:
    <div>
    <img src="images\charucoboard.png" width="300"/>
    </div>

    [Image of a ChArUco board](https://docs.opencv.org/3.4/df/d4a/tutorial_charuco_detection.html)

    To calibrate using a ChArUco board, it is necessary to detect the board from different viewpoints.
    The ChArUco corners and ChArUco identifiers captured on each viewpoint are then stored.
    The calibrateCamera() function will fill the camera_matrix and distortion_coeffs arrays with the camera calibration parameters, which is then stored in a json file. 

    Now, we can take an image and undistort it. However first, we can refine the camera matrix using cv2.getOptimalNewCameraMatrix(), and then show the undistorted frame. 

* ##### **Marker Detection**

     
     
     - Principle:
    
        The principle of the marker detection used by OpenCV is explained in detail, [here](https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html).

        When using the `detectMarkers()`method of the ArucoDetector class we use the predefined default detector parameters. Here we want to explain what these parameters are to better understand how the markers are so efficiently detected.

        The first parameters concern the adaptive thresholding of the frame, to determine how many thresholding steps are performed onto the frame as well as the minimum and maximum adaptive Threshold Window sizes (see OpenCV [adaptiveThreshold()](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3) function)

        After finishing the thresholding step, the contours of the markers are determined. At this stage contours for non marker candidates are filtered out in multiple steps. Afterwards, the bit of each candidate is analysed to determine whether or not they are actually a marker and of course if they are a marker of the marker dictionary defined beforehand.  
        
        
     
    - Usage:
    
        We first define a dictionary of our 6 markers with cv2.aruco.extendDictionary() function
        The tuples `marker_corners` and `marker_ids` are then assumed to be directly obtained from cv2.aruco.ArucoDetector.detectMarkers(), at each sampling time.

        `marker_corners` is a tuple of arrays defining every 4 corner position for each ArUco marker, in their original order (which is clockwise starting from the top left).

        `marker_ids` is the list of ids of each marker, to characterize each one.

        The markers with IDs 0 through 3 are defined as the map corners in clockwise order, starting from the top left. The markers 4 and 5 are respectively attributed to the robot and the target.

        Taking advantage of the ordered corners and their given position, for each marker, we can compute the position & orientation of the Thymio using the function `detect_robot()` and the position of the targetwith `detect_target()`.  It must be noted that a marker can be found rotated in the environment, however, the detection process needs to be able to determine its original rotation, so that each corner is differentiated and identified.


* ##### **Image correction**
     
     
     In order to build a map from the camera frame, we need to perform two transformations:
     - First of all, we need to correct the lens distortions, by applying a cv2.undistort on the camera frame. The calibration parameters (namely the camera matrix and the distortion coefficients) are read from a json file, which is created by running **src/calibrate_camera.py**.
    - Secondly, we need to correct the perspective of the undistorted frame. By using the four detected corners of the map, and the corresponding four corners of the map image, we can compute a perspective transformation matrix using cv2.getPerspectiveTransform. cv2.warpPerspective then allows us to get a perspective correct frame, where each corner is at a marker corner.

    This undistorted, perspective corrected frame can be directly used by the map generation algorithm to extract the obstacles, and to detect the position of the robot and the target.

### II. Global navigation

* #### **Parameters**

These are the global constants representing the different dimensions of the robot and its environment.

| Name                  | Value                          |
|-----------------------|--------------------------------|
| FRAME_WIDTH           | 960                            |
| FRAME_HEIGHT          | 720                            |
| MAP_WIDTH_MM          | 1334                           |
| MAP_HEIGHT_MM         | 968                            |
| MAP_WIDTH_PX          | 900                            |
| MAP_HEIGHT_PX         | Computed from map aspect ratio |
| SAMPLING_TIME         | 0.1                            |
| MMS_PER_MOTOR_SPEED   | 0.4348                         |
| TARGET_RADIUS_MM      | 70.0                           |
| MARKER_SIZE_MM        | 57.0                           |
| ROBOT_RADIUS_MM       | 80                             |
| ROBOT_CENTER_TO_WHEEL | 48                             |
| ROBOT_WHEEL_RADIUS    | 22                             |
| ROBOT_WHEEL_SPACING   | 96                             |
| DILATION_RADIUS_MM    | ROBOT_RADIUS_MM + 25           |
| ROBOT_MASK_RADIUS_MM  | ROBOT_RADIUS_MM + 20           |
| TARGET_MASK_RADIUS_MM | TARGET_RADIUS_MM + 10          |
| MARKER_MASK_SIZE_MM   | MARKER_SIZE_MM + 5             |

* #### **Obstacle mask extraction**
     
     
     The extraction of the obstacles from the perspective corrected frame is done in five main steps:  

    - First of all, the grayscale frame is thresholded by a fixed value (of 120). We use an inverse binary threshold, such that all areas that were darker than the threshold become '1' (represents obstacles), and all areas that were lighter than the threshold become '0' (represents free space).
    - Secondly, the robot, target and corner markers of the map are masked out from the obstacle mask, as to not be considered as obstacles.
    - Then, we use a cv2.MORPH_OPEN morphology operation with a small kernel (5x5), in order to discard tiny obstacle pixel groups (in the range of a few millimeters). This could be dirt on the map, folds in the paper making a sharp shadow, etc. Note that cv2.MORPH_OPEN is just erosion followed by dilation.
    - We then add an artificial 1 pixel wide border around the map, to keep the robot fully within the bounds.
    - Finally, we dilate the obstacle mask by the robot radius converted to pixels, plus a 25mm margin to allow for uncertainties and errors.

    The function **get_obstacle_mask** in **image_processing.py** implements this algorithm.
    
    We show below all the steps of obstacle extraction, from camera frame to final free space mask.

    ![](images/camera_frame.png)

  *Frame from the camera (already undistorted in this capture)*

    ![](images/obstacle_extraction/warped.png)

  *Perspective corrected frame*

    ![](images/obstacle_extraction/thresholded.png)

  *First raw obstacle mask: thresholded image*
    
    ![](images/obstacle_extraction/masked.png)

  *Masked robot, target and corner markers*

    ![](images/obstacle_extraction/opened.png)

  *After the MORPH_OPEN operation*

    ![](images/obstacle_extraction/borders.png)

  *Adding borders*

    ![](images/obstacle_extraction/dilated.png)

  *Dilation*

* #### **Obstacle contours extraction**
     
     
     When we have a mask of the obstacles, we can move on to finding their contours.  
     The first step here is to invert the mask, such that a '1' represents free space. The outlines of these zones of free space are extracted using the cv2.findContours function of OpenCV. Note that we only use cv2.CHAIN_APPROX_SIMPLE as an option, which means series of points lying on horizontal, vertical or diagonal lines will be combined. The number of vertices we get at the end is still in the thousands though, and untractable for a visibility graph.  
     
     The solution is to approximate the contours by a polygon of lower vertex count, using cv2.approxPolyDP. We found that an epilon of 2 pixels gave good results, lowering the total number of vertices to a few hundred at max, while keeping a good approximation of sharp features in the obstacle mask.  
     
     The findContours function also gives their topological hierarchy, which allows us to group them by "regions", which are disconnected regions of the map (i.e. there is no possible path from one to the other). In the subsequent steps where we extract the visibility graph, we can perform checks only within a region, which increases performance. Note that this is hard to demonstrate with the Thymio due to the available size of the map and the need to slit it in two (at least), but it would extend very well to a broader mapped space.  
     
     The function extract_contours from **global_map.py** implements the steps explained above.

* #### **Visibility graph creation**
     
     
     The visibility graph is created in two steps:

    - First of all, the static part of the graph if built (i.e. only edges between obstacle vertices). This is the slowest operation, taking approximately 1 second for the kind of map we tested.
    - Secondly, the dynamic part of the graph is built (i.e. the edges between the robot or the target and the obstacle vertices).

    About the implementation of the algorithm, two important considerations allow us to get much better performance than a naive approach:
    
    - Only convex vertices need to be considered. Given any two random vertices on the map, the shortest path between them will never pass by a concave vertex. Geometrically, if we think about a string going around a spiky shape, it will always only touch convex vertices.
    - Assume we are considering a given convex vertex, trying to decide with which other ones to consider an edge or not. If the considered vertex has its two neighbors on the contour lying on each "side" of the edge, we do not need to include him. Geometrically, any path that would pass by this vertex would always be shorter by going directly to one of its neighbors instead.

    When the set of remaining edges, we must make sure they are free of any intersection with all others (which is `O(N^2)`), hence the importance of the pre-filtering.

    Also note that the source and target vertices are first projected out of obstacles if they happen to be inside one. This means there will always be a path even if the robot is within the obstacle mask (the red zone). This increases robustness, as the path following controller will always provide an input even if the robot is slightly in the obstacle zone. This is required in the case of a visibility graph, because we are following the borders of obstacle zones, which means we might be within that zone a significant portion of the time.

* #### **Pathfinding**
     
     
     When we have the full visibility graph, we use a simple Dijkstra's algorithm to find the shortest path between the source (the robot) and the target. Since the total number of edges in the graph is very low (can be less than 100 for most environments), there is no point in implementing a greedier search such as A*.

    ![](images/global_map.png)
    *Global map creation and pathfinding on a test image. This is a capture of the intereactive visualization below.*

* #### **Interactive map**
     
     
     The full implementation of the map creation from obstacle mask to pathfinding can be interactively tested by running the code below. You can click anywhere to place the source point, and the target is the point under the mouse cursor. This visualization allows to see the vertices of the contours (black being the first one, and white the last one), as well as the visibility graph and the shortest path.

In [ ]:
import sys
sys.path.insert(0, './src')

from global_map import *
from image_processing import *

g_target = (120, 730)
g_source = (200, 100)


def draw_contour_orientations(img: np.ndarray, contours: list[np.ndarray]):
    """
    Draw positive orientation as green, negative as red;
    first vertex is black, last is white
    """
    for c in range(len(contours)):
        orientation = np.sign(cv2.contourArea(contours[c], oriented=True))
        color = (64, 192, 64) if orientation >= 0 else (64, 64, 192)
        cv2.drawContours(img, [contours[c]], contourIdx=-1, color=color, thickness=2)
        n_points = len(contours[c])
        for i in range(n_points):
            brightness = i / (n_points - 1) * 255
            cv2.circle(img, contours[c][i], color=(brightness, brightness, brightness), radius=5, thickness=-1)


def interactive_map_and_path():
    color_image = cv2.imread('images/map_divided.png')
    obstacle_mask = get_obstacle_mask(color_image, robot_position=None, target_position=None, mask_corner_markers=False)

    approx_poly_epsilon = 2
    regions = extract_contours(obstacle_mask, approx_poly_epsilon)

    all_contours = [contour for region in regions for contour in region]

    free_space = np.empty_like(color_image)
    free_space[:] = (64, 64, 192)
    cv2.drawContours(free_space, all_contours, contourIdx=-1, color=(255, 255, 255), thickness=-1)

    graph = build_graph(regions)

    cv2.namedWindow('main', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('main', color_image.shape[1], color_image.shape[0])

    while True:
        free_source, free_target = update_graph(graph, regions, np.array(g_source), np.array(g_target))
        path = dijkstra(graph.adjacency, Graph.SOURCE, Graph.TARGET)

        img = cv2.addWeighted(color_image, 0.75, free_space, 0.25, 0.0)
        draw_contour_orientations(img, all_contours)
        cv2.drawContours(img, all_contours, contourIdx=-1, color=(64, 64, 192))
        draw_graph(img, graph)
        draw_path(img, graph, path, np.array(g_source), free_source, np.array(g_target), free_target)

        cv2.namedWindow('main', cv2.WINDOW_NORMAL)
        cv2.setMouseCallback('main', mouse_callback)
        cv2.imshow('main', img)
        if cv2.waitKey(1) == 27:
            break

    cv2.destroyAllWindows()


def mouse_callback(event, x, y, flags, param):
    global g_target, g_source
    if event == cv2.EVENT_MOUSEMOVE:
        g_target = (x, y)
    if event == cv2.EVENT_LBUTTONDOWN:
        g_source = (x, y)


interactive_map_and_path()

### III. Extended Kalman Filter

![](gif/camera_hidden.gif)

Kalman Filters are very useful in the case of location estimation. The Kalman filter uses a system model along with measurements and system inputs, to maintain an estimate on our position and orientation.
Our filter is based on an odometric model using the motor speed readings as inputs, and the geometry of the robot movement. The Kalman filter relies on the measurements of the position & orientation provided by the camera, to correct the estimate. The state variables of interest here will be the coordinates $x$, $y$ and $\theta$ defined as in the figure:


<div>
<img src="images\Thymio_schema.jpeg" width="300"/>
</div>

Schematic of the Thymio reference used


We therefore have the state vector $\textbf{x} = (x,y,\theta)$.
Readings are obtained at a sampling time of $T_s = 0.1 \, \text{s}$ and the coordinates are updated by:
$$\begin{bmatrix}
    x_{k+1} \\ 
    y_{k+1} \\ 
    \theta_{k+1}
\end{bmatrix} = \begin{bmatrix}
    x_{k} \\ 
    y_{k} \\ 
    \theta_{k}
\end{bmatrix} + \begin{bmatrix}
    \frac{\omega_{r,k}+\omega_{l,k}}{2}  R_w T_s \sin\left({\theta_k+\frac{1}{2}\frac{\omega_{r,k}-\omega_{l,k}}{d} R_w T_s}\right) \\ 
    \frac{\omega_{r,k}+\omega_{l,k}}{2}  R_w T_s \cos\left({\theta_k+\frac{1}{2}\frac{\omega_{r,k}-\omega_{l,k}}{d} R_w T_s}\right) \\ 
    2 \cdot \frac{1}{2}\frac{\omega_{r,k}-\omega_{l,k}}{d} R_w T_s
\end{bmatrix}$$

Here $R_w$ is the wheel Radius and d is the space between the wheels.

We can further simplify these equations by defining:

`tangential_speed`: $\delta V_k = \frac{\omega_{r,k}+\omega_{l,k}}{2}  R_w T_s$

`delta.angle`: $\delta \theta_k = \frac{\omega_{r,k}-\omega_{l,k}}{d} R_w T_s$





In our case the sate is observed by the camera measurements of $x$, $y$ & $\theta$ with an independant Gaussian noise process $v$.
Therefore our mobile robot can be defined by:
$$x_k = g(x_{k-1},u_k) + w_k $$
$$y_k = h(x_{k-1}) + v_k $$
Here, the function $g$ defines our motion model with $ w_k \sim N(0, Q)$ is a multivariate normal distribution that models the uncertainty introduced by the state transition. The function $h$ defines our measurement model $v_k \sim N(0, R)$ is a multinormal describing the measurement noise. Both $w_k$ and $v_k$ are considere as zero-mean white Gaussian Noise, with $Q$ and $R$ as, respectively, the constant covariance of the process noise and the measurements.



Denoting our nonlinear function for the motion model $g$, we used an Extended Kalman Filter. In the case of well defined transition models, the EKF has been considered the de facto standard in the theory of nonlinear state estimation, navigation systems and GPS.

* #### **Prediction step**
     
     
     The prediction step consists of computing the mean and covariance of the predicted state using linearization of the motion model $g(x_{k-1},u_k)$.  
     
    The a priori estimated mean of our state is defined by:
    $$\bar{\mu}_k = g(x_{k-1},u_k)$$  
    
    Furthermore, the a priori estimated covariance of the state is:
    $$\bar{\Sigma}_k = G_k \Sigma_{k-1} G_k^T + Q$$  
    
    Here $G$ is defined as the Jacobian of the motion model $g$:
    $$  G_k = \left.\frac{\partial g}{\partial \textbf{x}_k}\right\vert_{\textbf{x}_k = \bar{\mu}_k, w_k = \textbf{0}} =  \begin{bmatrix}
    1 & 0 & \delta V \cos(\theta_k + \delta \theta_k)\\
    0 & 1 & -\delta V \sin(\theta_k + \delta \theta_k) \\
    0 & 0 & 1 \\
    \end{bmatrix}$$

    We also need to define the constant covariance matrix $Q$ of the process noise. We assumed the standard deviation of on the state position variables $x$ and $y$ to be approximately of 1 millimeter and for the angle $\theta$ to be of $0.04 \, \text{radians}$.   
    Therefore obtaining:
    
    $$Q = \begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 0.04 \\
    \end{bmatrix}$$  
    
    This covariance matrix was mostly determined through trial and error, tuning the parameters along to obtain the desired result. We did experiment with a relatively high initial covariance and kept the final result at which it converges to, keeping only the diagonal values.

* #### **Measurement Update**
     
     
     The next step is updating our estimation from the previous step, using the measurements to compute the a posteriori mean and covariance of our estimated state. The goal is to improve the accuracy of the estimated state by incorporating the real-world measurements. We first need to determine the difference between the camera measurements and our a priori estimate, this is the innovation $i$:
    $$i = y_k - \bar{\mu}_k$$  
    
    From this, we can determine the measurement prediction covariance $S$ by adding our Measurement covariance matrix $R$:
    $$S_k = \bar{\Sigma}_k + R$$  
    
    Which allows us to compute the Kalmain Gain $K$ which specifies the degree to which the measurement is incorporated into the new state estimate. It is computed so to minimize the a posteriori error covariance:
    $$K_k =  \bar{\Sigma}_k S_k^{-1}$$  
    
    Finally we obtain our new a posteriori estimate with mean $\mu_k$ and covariance $\Sigma_k$:
    $$\mu_k = \bar{\mu}_k + K_k i$$
    $$\Sigma_k = \bar{\Sigma}_k - K_k \bar{\Sigma}_k $$  
    

    We also need to define our constant measurement covariance R, we determined to be:
    $$R = \begin{bmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 0.04 \\
    \end{bmatrix}$$  
    

    The $R$ matrix was more or less defined in the same manner as the $Q$ matrix through tuning the parameters until we got a satisfying result.

* #### **Filtering without the camera measurements**
     
     
     When updating our estimation using the measurements, we need to first consider whether or not these measurements are available. If the camera were to be disturbed or if one of the ArUco markers were obstructed from view, our estilation can no longer rely on these camera measurements.

    We implemented this potential scenario by setting $y_k$ as the a priori estimate $\bar{\mu}_k$. This makes the innovation null and therefore cancelling out the effects of the Kalman gain on the mean of our a posteriori estimate whilst still adjusting the covariance.

* #### **Implementation**
     
     Inputs
    - `measurements`: (Optional) np.ndarray containing the position $(x,y, \theta)$  of the robot
    - `mu_km`: Previous state estimation mean at time step $(k-1)$
    - `sig_km`: Previous state estimation covariance at time step $(k-1)$
    - `speed_left`: Left motor speed input of the Thymio
    - `speed_right`: Right motor speed input of the Thymio
      
    Outputs
    - `x_est`: New state estimation mean at time step $k$
    - `sig_est`: New state estimation covariance at time step $k$

In [ ]:
def kalman_filter(measurements: Optional[np.ndarray], mu_km: np.ndarray, sig_km: np.ndarray, speed_left: float,
                  speed_right: float) -> tuple[np.ndarray, np.ndarray]:
    # Since our discretized motion model does not perfectly match reality, try to correct it by some extent.
    # Since forward Euler integration tends to extrapolate "too far" for smooth trajectories, artificially reduce the
    # contribution of the derivative to the state update. Note that this is just a "hack" that seems to work well enough
    # for this system, and the specific multiplication factor was found through experimentation. For a more precise
    # motion model, we should use a better integration scheme (Runge-Kutta 4 for example), and move the Kalman filter
    # computation to the hardware, to get significant lower latencies on the inputs, as well as using a higher sampling
    # frequency.
    EFFECTIVE_SAMPLING_TIME = SAMPLING_TIME * 0.8

    tangential_speed = (speed_left + speed_right) / 2
    angular_speed = (speed_right - speed_left) / ROBOT_WHEEL_SPACING
    delta_angle = angular_speed * EFFECTIVE_SAMPLING_TIME
    # Prediction through the a priori estimate
    x_est = mu_km[0]
    y_est = mu_km[1]
    angle_est = mu_km[2]
    # Estimated mean of the state
    mu_k_pred = np.zeros(3)
    mu_k_pred[0] = x_est + tangential_speed * EFFECTIVE_SAMPLING_TIME * -np.sin(angle_est + delta_angle)
    mu_k_pred[1] = y_est + tangential_speed * EFFECTIVE_SAMPLING_TIME * np.cos(angle_est + delta_angle)
    mu_k_pred[2] = angle_est + delta_angle

    # Jacobian of the motion model
    G_k = np.eye(3)
    G_k[0, 2] = -tangential_speed * np.cos(angle_est + delta_angle)
    G_k[1, 2] = -tangential_speed * np.sin(angle_est + delta_angle)

    # Estimated covariance of the state
    sig_k_pred = G_k @ sig_km @ G_k.T
    sig_k_pred += KALMAN_Q

    if measurements is not None:
        y = measurements
    else:
        # If no measurements we consider our measurements to be the same as our a priori estimate as to cancel out the
        # effect of innovation
        y = mu_k_pred

    # Innovation / measurement residual
    i = y - mu_k_pred

    # Correctly handle the case where the angle difference is discontinuous
    if i[2] < -np.pi:
        i[2] = 2 * np.pi + i[2]
    elif i[2] > np.pi:
        i[2] = - 2 * np.pi + i[2]

    # Measurement prediction covariance
    S = sig_k_pred + KALMAN_R

    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = sig_k_pred @ np.linalg.inv(S)

    # A posteriori estimate
    x_est = mu_k_pred + K @ i
    sig_est = sig_k_pred - K @ sig_k_pred

    # Keep the angle within [-pi, pi]
    if x_est[2] < -np.pi:
        x_est[2] = 2 * np.pi + x_est[2]
    elif x_est[2] > np.pi:
        x_est[2] = - 2 * np.pi + x_est[2]

    return x_est, sig_est

### IV. Motion Control

![](gif/follow_path.gif)

* #### **Implementation method**  
    For the control of the movement of the robot, we use the **Astolfi** controller seen in the course. Using this controller, we get a smooth trajectory betweeen the different points of the path to the goal.  

    - **Inputs :**
        - *state* : corrected (or predicted if the camera is hidden) state given by the Kalman filter
        - *goal_state* : position of the next goal on the global path  
        
    - **Outputs :**
        - *u_r* : speed of the right wheel of the robot (in mm/s)
        - *u_l* : speed of the left wheel of the robot (in mm/s)
        - Boolean switch : **TRUE** if the goal is attained, **FALSE** if not

* #### **Differences with the course**  
     
     
     The following image represents the configuration for which the Astolfi controller is defined in the course :  
     
     ![astolfi_course](images\astolfi_course.png)
     
     For our implementation, we defined the orientation of the robot differently : instead of having an orientation of 0 degrees when looking in the direction *X*, we decided that the robot had an orientation of 0 degrees when looking in the direction *Y*.  
     As seen on the image below :
     ![robot_orientation](images\robot_orientation.png)  
     
     Hence, the *X* axis defined the course becomes our *Y* axis, and the *Y* axis defined in the course becomes our *-X* axis. This has an impact on the way that the reference angle is calculated (absolute angle made by the normal (our axis *Y*) leaving from the position of the robot and the segment between the position of the robot and the position of the goal).  
     ![arctan](images\arctan.png)  
     
     As the *Y* axis of the course becomes our *-X* axis, we can compute the reference angle as :  
     $ \theta_R = arctg(\dfrac{-\Delta x}{\Delta y}) $
     
     

* #### **Code implementation**  


In [ ]:
from parameters import *


def astolfi_control(state, goal_state) -> tuple[float, float, bool]:
    """
    Returns the left desired speed, right desired speed and whether the goal was reached
    """

    Kp = 8  # > 0
    Ka = 75  # > Kp
    goal_radius = 20

    delta_x = goal_state[0] - state[0]
    delta_y = goal_state[1] - state[1]
    reference_angle = -np.arctan2(delta_x, delta_y)

    rho = np.sqrt(delta_x ** 2 + delta_y ** 2)
    alpha = reference_angle - state[2]
    if alpha < -np.pi:
        alpha = 2 * np.pi + alpha
    elif alpha > np.pi:
        alpha = - 2 * np.pi + alpha

    if rho <= goal_radius:
        return 0, 0, True

    power = 1/4
    K_power = 140 / (goal_radius**power)
    v = Kp * K_power * rho**power
    # v = Kp * rho
    omega = Ka * alpha

    u_r = (ROBOT_CENTER_TO_WHEEL * omega + v) / ROBOT_WHEEL_RADIUS
    u_l = (v - ROBOT_CENTER_TO_WHEEL * omega) / ROBOT_WHEEL_RADIUS

    speed_threshold = 80
    u_r = np.clip(u_r, -speed_threshold, speed_threshold)
    u_l = np.clip(u_l, -speed_threshold, speed_threshold)

    return u_l, u_r, False

* #### **Gain tuning**  
     
     
     We have the following speeds with the corresponding gains for the Astolfi controller defined in the course :  
     ![astolfi_speeds](images\astolfi_speeds.png)  
     We don't want to force the angle **$\beta$** to go to zero (we don't want the robot to be vertical each time it reaches a goal) so we set the value of the **Kb** (in the picture above $k_\beta$) parameter really close to zero.  
     
     The gain controlling the orientation and the angular error of the robot **Ka** (in the picture above $k_\alpha$) was tuned after testing multiple values and choosing the one that we thought had the best effect.  
     
     For the linear speed of the robot $v$, we modified a little bit the Astolfi controller from the course. Instead of having the linear speed of the robot being linearly proportional to the distance to the goal, $v$ is now proportional to the fourth root of the distance to the goal. In doing this, the linear speed only drops to a really small value when the robot is really close to the goal instead of dropping linearly when getting close to the goal. This behavior can be seen in the figure below.  
     ![speed_diff](images\speed_diff.png)  
     As seen in the picture above, when we get to close to the goal radius (which we defined as 20mm), the speed using our modified controller is way higher than the speed of the controller given in the course. This allows for smoother transitions between goals and an overall smoother trajectory.
     

* #### **Kidnapping**

    Kidnapping works very simply: as soon as the robot or the target moves too much from one frame to the next, we recompute the path.

  ![](gif/kidnapping.gif)

### V. Local navigation

![](gif/local_avoidance.gif)

Efficient local navigation is an essential element in the field of autonomous robotics, particularly for mobile robots operating in partially known environments. This strategic approach combines global navigation methods to find a viable path leading to the goal with local navigation mechanisms to avoid obstacles in real time.

Let's take the example of an autonomous car: it can't just go straight to the goal without taking into account what's happening in its environment, such as other cars on the road or pedestrians crossing the road.  
It has to be able to deal with these local obstacles.  

The same applies to our Thymio: it needs to be able to switch from global to local navigation in the event of new local obstacles being detected.  
In our case, the switch from local to global is made via our counter **num_samples_since_last_obstacle**.

If we're in the global case, our robot will take the speed values returned by our controller, but if we're in our local navigation, the robot's behavior becomes independent of global navigation until our counter reaches a value greater than 3, so until a certain time when we haven't seen an obstacle, at which point the controller takes back control of our robot.

We decided to use this attribute to give the robot time to move forward after avoiding an obstacle, before switching back to global navigation to avoid the undesirable effect of our controller. Indeed, it will tend to return to the path and hit the obstacle if it is between the robot and the path, before detecting it due to the radical movement of the controller's commands.

See implementation details below or directly in the **main.py** file, in the function **run_navigation**.

* #### Parameters

|   Name   |   Initial value  |  Description |  
|---    |:-:    |:-:        |
|   motor_speed   |   200   |   Reference speed of the robot for avoidance | 
|   threshold   |   17  |   Sensor threshold for obstacle detection | 
|   prox_gain   |      0.07 |  Gain by which we multiply the sensor values to define the speed at which we turn to avoid the obstacle  | 
|   num_samples_since_last_obstacle   |   -1    | Counter to switch from local to global navigation |  
|   case   |   None   |Integer varying if the obstacle is to the left, to the right, in front of the robot, or even if there is no obstacle |  


* #### Implementation

In [ ]:
#Speed command given by our controller
speed_left_target = np.clip(int(input_left / MMS_PER_MOTOR_SPEED), -500, 500)
speed_right_target = np.clip(int(input_right / MMS_PER_MOTOR_SPEED), -500, 500)

# Do local obstacle detection
prox_horizontal = [0] * 5
for i in range(5):
    prox_horizontal[i] = list(nav.node.v.prox.horizontal)[i]
    
# The "avoid obstacle" function will modify and return the counter value; if it detects an obstacle, 
#it will reset the counter to zero, transitioning to local navigation accordingly.
nav.num_samples_since_last_obstacle = aw(
    avoid_obstacles(nav.node, nav.num_samples_since_last_obstacle, prox_horizontal))

# If we recently saw an obstacle (i.e. more recently than 3 SAMPLING_TIME ago), do not yet switch to path
# following, as to hopefully let the robot clear the obstacle.
if 0 <= nav.num_samples_since_last_obstacle <= 3:
    nav.num_samples_since_last_obstacle += 1
elif nav.num_samples_since_last_obstacle > 3:
    nav.num_samples_since_last_obstacle = -1

# If no obstacle was recently detected, send the controller's desired speeds to the motors
elif nav.num_samples_since_last_obstacle == -1:
    nav.node.send_set_variables(set_motor_speed(speed_left_target, speed_right_target))

* #### **Obstacle detection**  
     
     
     Ideally, the robot should have access to its local environment at all times, so that it can anticipate obstacle avoidance, especially if the obstacle is moving, and react to it at all times.    
     In discrete time, this proves complicated, so we need to set a sampling time for which we retrieve the data. Here, our sampling time is 0.1s.    
     We therefore have access to the sensor values every 0.1s, which enables us to react according to the read values.  
       
     In our program, obstacle detection is performed using our thymio's 5 proximity sensors.  
     These sensors use the infrared emission/reception principle, they measure the distance towards an obstacle by generating an infrared signal, and the particular electronics of the horizontal sensor filters the ambient light.  
       
     We set a threshold defining the detection of an obstacle, and compare this threshold with the values of our sensors. Once this threshold is exceeded, we consider that an obstacle has been detected, and then consider different cases.  
     The values of the proximal sensors range from 0 (the robot doesn't detect anything) to several thousand (the robot is very close to an obstacle), knowing that we decide to set the threshold to 17.  
       
     See details below or directly in the **local_navigation.py** file in the **avoid_obstacle** function.
  


In [ ]:
# If an obstacle is detected, we switch to obstacle avoidance mode instead of goal tracking mode.
if ((obst[0] > HTobst or obst[1] > HTobst) and (obst[0] > obst[2] or obst[1] > obst[2])):
    # Different cases are distinguished
    # Case 0 where the obstacle is on the left
    num_samples_since_last_obstacle = 0
    case = 0

elif ((obst[4] > HTobst or obst[3] > HTobst) and (obst[4] > obst[2] or obst[3] > obst[2])):
    # Case 1 where the obstacle is on the right
    num_samples_since_last_obstacle = 0
    case = 1

elif (obst[2] > HTobst and (obst[0] < obst[2] or obst[4] < obst[2]) and (obst[1] <= obst[2] or obst[1] <= obst[2])):
    # Case 2 where the obstacle is in front
    num_samples_since_last_obstacle = 0
    case = 2
else:
    #In this case we didn't detect obstacles
    case = 3

* #### **Obstacle avoidance**  
     
     
     Once the robot has detected potential obstacles, it can avoid them depending on the specific case.  
       
     In the case where the obstacle has been detected on the sides, the robot will avoid it by turning in the opposite direction of the obstacle.  
     In the case where the obstacle is in front of the robot, it will move backwards and turn in a random direction to avoid the obstacle.  
     And in each case, the robot will move straight until the counter reaches a value greater than 3 or detects another obstacle.  
       
     See details below or directly in the **local_navigation.py** file in the **avoid_obstacle** function.
     

In [ ]:
if case == 0 or case == 1:
    # Obstacle is to your left or right.
    # Obstacle avoidance: accelerate the wheel near the obstacle to turn in the opposite direction to the obstacle.
    if case == 0:
        base_speed = prox_gain * (prox[0] + prox[1])
        await move_forward(node, int(base_speed), int(base_speed * 0.1))

    elif case == 1:
        base_speed = prox_gain * (prox[4] + prox[3])
        await move_forward(node, int(base_speed * 0.1), int(base_speed))

elif case == 2:
    # Obstacle is in front of the Thymio.
    # Avoid the obstacle in front by backing up and then turning randomly on one side.
    if np.random.randint(2) == 0:
        await step_back(node, motor_speed)
        await turn_left(node, motor_speed)
    else:
        await step_back(node, motor_speed)
        await turn_right(node, motor_speed)

elif num_samples_since_last_obstacle >= 0:
    await move_forward(node, motor_speed, motor_speed)

* #### **Example of implementation**  
     
     
     ![speed_diff](gif\Local_avoidance.gif)  
     In this example, we can see that our robot detects our obstacle, avoids it by turning in the opposite direction, keeps going straight ahead until our counter **num_samples_since_last_obstacle** reaches a value of 3, then the controller and global navigation take over, guiding our robot to the goal.    
     

### VI. Interface

![](gif/follow_path.gif)

We developped a simple but expressive interface to be able to visualize the whole system while it is running. The three windows include:
- The frame from the camera, with the detected map corners drawn as red crosses.
- The perspective corrected frame with the obstacle mask, the map graph and the path. We also draw on this image the pose of the robot as seen by the camera (in red), and as estimated by the Kalman filter (in green). The detected target position is also drawn in green.
- Plots of the estimated and measured X and Y position of the robot, and a plot of the estimated and measured angle of the robot.

On the map window, the button "Build graph" (re-)builds the obstacle mask and visibility graph. As soon as the graph changes or the position of the robot or the target varies abnormally, the path is re-computed. The button "GO!" starts the path following.

On the plots window, the number of plotted samples can be changed with a slider, and the axes auto-fit can be disabled if necessary.

The interface was coded using [DearPyGui](https://github.com/hoffstadt/DearPyGui), a very powerful and simple to use graphical interface library based on [Dear ImGui](https://github.com/ocornut/imgui).

### VII. Final program

The final full program can be executed by simply running src/main.py, or alternatively by running the code cell below.

Important notice for the user:
- The program expects to find a camera.json file in the same directory from which it is run. This file is included in the src/ folder alongside the rest of the code, and corresponds to the last calibration of our camera. It is created by running the file calibrate_camera.py, as explained in the section Camera calibration.
- The present version of the code opens the video capture with the DirectShow backend, which was required to be able to change the camera resolution on the test PC. This would obviously need to be changed if run for example on a Mac. There sadly seems to be no VideoCapture setting that allows it to be used everywhere without modification.

In [ ]:
import sys
sys.path.insert(0, './src')

from main import main

main()

### **VII. Difficulties encountered**

* #### **Angle error discontinuity**
  
  When calculating the angle error between the orientation of the robot and the reference orientation defined by the goal, we use the **numpy** angular function *arctan2* which returns a value in the range [$-\pi$, $\pi$]. In our Kalman filter and motion control implementations, this led to wrong behaviors when the error would go just past $-\pi$ or $\pi$ as the error value would suddenly jump from a big negative value to a big positive value (or the other way around).  
  
  We corrected this with the following piece of code that we added to our Kalman filter and motion control implementations :

In [ ]:
# Correctly handle the case where the angle difference is discontinuous

if i[2] < -np.pi:
    i[2] = 2 * np.pi + i[2]
elif i[2] > np.pi:
    i[2] = - 2 * np.pi + i[2]

* #### **Change of controller** 
     
     
  We first settled on the use of a PID controller for the implementation of the motion control of our robot. With this controller, the robot would turn towards the goal, once its orientation is equal to the angle defined by the goal plus or minus some angular threshold, it would start to make its way towards the goal. Then once it is in a certain region around the goal, it would switch its objective to the next goal and reiterate the steps. This implementation, although simple at first sight, presented quite a lot of issues for us :  
  
     - If we set the angular threshold too low, the robot would sometimes never attain the desired orientation and just oscillate around the reference angle.
     - If we set the angular threshold too high, the robot would deviate too much from its path to the goal and thus never attain it
      - When set on the right path, our robot would not go straight when given same inputs for the speeds of the right and left wheels, which made it deviate from its desired path
        
    In the end, we decided to abandon this solution and implement the Astolfi controller which worked a lot better.

* #### **Local navigation implementation with the Astolfi controller**
     
     
  When first implementing our local navigation process, we had issues to make it work as the Astolfi controller that we used would immediately rectify the robots direction to go back to its original path and thus hit the obstacle.  
  
  To make this work, we made the robot first avoid the obstacle then go straight for some time before going back into the motion control process. This way, the robot has enough room to go past the obstacle and then go back to its original path.

* #### **Computer vision with colors points**     
        
     
 
   At first, we tried to define the environment using points of different color (for example green for the corners of the map, red for the thymio and blue for the goal). Although this worked with good lighting conditions, we quickly found out that it was really dependent of the lighting and thus of where we placed the environment (this worked differently when in the Polydome or when in the CO building). Because of this, we had to tune some parameters or recalibrate the computer vision each time we changed locations or the lighting changed.  
     
     To resolve this issue, we decided to use **ArUco** markers to define the corners of the map as well as the positions of the Thymio and the goal. This works way better than color points and is most importantly way more robust when it comes to lighting conditions.

* #### **Be careful with the units**  
     
 
     
  We had quite a number of issues when putting all the systems together because of the use of different units for the different modules of the project. For example, the motion control used millimeters while the Kalman filter used meters at first. Or the computer vision used radians while the motion control used degrees for the angles. This was an easy fix but that still made us lose a bit of time.